In [ ]:
#importing all the required packages

from PIL import Image
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout,Activation
from keras.utils.np_utils import to_categorical
from tensorflow.keras.models import Sequential
from keras.preprocessing import image
import tensorflow as tf 
import matplotlib.pyplot as plt
import pickle as pkl

In [ ]:

     #loading all images to a dictionary
images_dir1_dict = {os.path.splitext(os.path.basename(x))[0]: 'E:\SEMESTER 6\Minor Project\ML Model And back End\HAM10000_images_part_1\\'+x
                     for x in os.listdir('E:\SEMESTER 6\Minor Project\ML Model And back End\HAM10000_images_part_1')}
images_dir2_dict2 = {os.path.splitext(os.path.basename(x))[0]: 'E:\SEMESTER 6\Minor Project\ML Model And back End\HAM10000_images_part_2\\'+x
                    for x in os.listdir('E:\SEMESTER 6\Minor Project\ML Model And back End\HAM10000_images_part_2')}
images_dir1_dict.update(images_dir2_dict2)

In [ ]:
#dictionary for describing names in short
disease_dict = {
        'akiec': 'Actinic keratoses',
        'bcc': 'Basal cell carcinoma',
        'bcc': 'Basal cell carcinoma',
        'bkl': 'Benign keratosis',
        'df': 'Dermatofibroma',
        'mel': 'Melanoma',
        'nv': 'Melanocytic nevi',
        'vasc': 'Vascular lesions'
}

In [ ]:
#loading the data to dataframe
data_df = pd.read_csv("F:\Projects\Detection-of-Skin-disease-using-CNN-main\HAM10000_metadata.csv")
data_df['path'] = data_df['image_id'].map(images_dir1_dict.get)
data_df['cell_type'] = data_df['dx'].map(disease_dict.get) 
data_df['cell_type_idx'] = pd.Categorical(data_df['cell_type']).codes


In [ ]:
data_df.head()

In [ ]:
pd.value_counts(data_df['dx'].values)

In [ ]:
#resizing
dims = (32, 32)
shape = dims + (3,)
data_df['image'] = data_df['path'].map(lambda x: np.asarray(Image.open(x).resize(dims)))

In [ ]:
y = data_df.cell_type_idx
x_train1, x_test1, y_train1, y_test1 = train_test_split(data_df, y, test_size=0.20)

In [ ]:
x_train = np.asarray(x_train1['image'].tolist())
x_test = np.asarray(x_test1['image'].tolist())


In [ ]:
#scaling
x_train = (x_train)/255
x_test = (x_test)/255

y_train = to_categorical(y_train1, num_classes = 7)
y_test = to_categorical(y_test1, num_classes = 7)

In [ ]:
plt.imshow(x_train[0])
plt.show()

In [ ]:
plt.imshow(x_test[0])
plt.show()

In [ ]:
#building CNN model
inp_shape = (32, 32, 3)
no_classes = 7
batch_size = 32 
epochs = 30

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=inp_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(no_classes, activation='softmax'))

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])



In [ ]:
model.summary()

In [ ]:
#providing dataset to model
history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1)

#Visualsing the model

In [ ]:
history.history

In [ ]:
acc = history.history['accuracy']

loss = history.history['loss']


epochs_range = range(epochs)

plt.figure(figsize=(15, 10))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')

plt.legend(loc='lower right')
plt.title(' Model Training  Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label=' Training Loss')

plt.legend(loc='upper right')
plt.title(' Model Training  Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()

In [ ]:
score = model.evaluate(x_test, y_test, verbose = 0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

In [ ]:
# with open('model.pkl', 'wb') as files:
#     pkl.dump(model, files)
model.save("model.h5")